In [17]:
import os, math

In [10]:
### Calculate travel time between two points: origin and destination
def traveltime(origin_id,destination_id,meters_per_minute,locations):
    dist=np.sqrt((locations.at[destination_id,'x']-locations.at[origin_id,'x'])**2\
                +(locations.at[destination_id,'y']-locations.at[origin_id,'y'])**2)
    tt=np.ceil(dist/meters_per_minute)
    return tt

# Read in data instance
A folder of data instance has 5 files: (1) couriers.txt, (2) instance_characteristics.txt, (3) instance_parameters.txt, (4) orders.txt, (5) restaurants.txt

In [55]:
# Read instance information
def read_instance_information(instance_dir):
    orders=pd.read_table(os.path.join(instance_dir,'orders.txt'))
    restaurants=pd.read_table(os.path.join(instance_dir,'restaurants.txt'))
    couriers=pd.read_table(os.path.join(instance_dir,'couriers.txt'))
    instanceparams=pd.read_table(os.path.join(instance_dir,'instance_parameters.txt'))

    order_locations=pd.DataFrame(data=[orders.order,orders.x,orders.y]).transpose()
    order_locations.columns=['id','x','y']
    restaurant_locations=pd.DataFrame(data=[restaurants.restaurant,restaurants.x,restaurants.y]).transpose()
    restaurant_locations.columns=['id','x','y']
    courier_locations=pd.DataFrame(data=[couriers.courier,couriers.x,couriers.y]).transpose()
    courier_locations.columns=['id','x','y']
    locations=pd.concat([order_locations,restaurant_locations,courier_locations])
    # locations.set_index('id',inplace=True)
    #
    # orders.set_index('order',inplace=True)
    # couriers.set_index('courier',inplace=True)
    # restaurants.set_index('restaurant',inplace=True)

    meters_per_minute=instanceparams.at[0,'meters_per_minute']
    pickup_service_minutes=instanceparams.at[0,'pickup service minutes']
    dropoff_service_minutes=instanceparams.at[0,'dropoff service minutes']
    target_click_to_door=instanceparams.at[0,'target click-to-door']
    pay_per_order=instanceparams.at[0,'pay per order']
    guaranteed_pay_per_hour=instanceparams.at[0,'guaranteed pay per hour']

    return orders,restaurants,couriers,instanceparams,locations

In [28]:
import pandas as pd
instance_dir = 'data/0o50t75s1p100'
orders,restaurants,couriers,instanceparams,locations,meters_per_minute,\
           pickup_service_minutes,dropoff_service_minutes,target_click_to_door,\
           pay_per_order,guaranteed_pay_per_hour=read_instance_information(instance_dir)

# Set hyper-parameters:
f∶ every f minutes solves a matching problem to prescribe the next pick-up and delivery assignment for each courier<br>
t : optimization time<br>
∆_(u): t+ ∆_(u ) is the assignment horizon<br>
∆_(1); ∆_(2): to determine Z_(t)<br>
beta: control the freshness in the construction of bundles
gamma: control the click to door time in the construction of bundles


In [14]:
# Accorting to default values in the paper
f = 5
delta_u = 10
delta_1 = 10
delta_2 = 10
# Not find in the paper
beta = 10 # should be tuned
gamma = 10 # should be tuned

# Procedure to initialize a solution

**Function that return U(t,r)**<br>
U(t,r) is the set of upcoming orders at restaurant r

In [67]:
def create_set_U_tr(t,r):
    ...

**Courier Object**

In [45]:
class Order:
    def __init__(self, order_information : dict):

        self.order_id = order_information.get('order')
        self.destination = (order_information.get('x'), order_information.get('y'))
        self.placement_time = order_information.get('placement_time')
        self.restaurant_id = order_information.get('restaurant')
        self.ready_time = order_information.get('ready_time')

In [110]:
class Courier(object):
    def __init__(self, courier_information : dict, restaurant : dict):
        self.courier = courier_information.get('courier')
        self.x = courier_information.get('x')
        self.y = courier_information.get('y')
        self.on_time = courier_information.get('on_time')
        self.off_time = courier_information.get('off_time')

        # Update and derived along the way
        self.travel = []
        self.visited = []
        self.orders = []
        # self.next_available_time = # a dictionary containing restaurant as key and available time as value
        self.current_position = (self.x, self.y)
        self.distance_travelled = 0
        self.current_time = self.on_time

        self.restaurant = restaurant

    @staticmethod
    def calculate_travel_time(source: tuple, restaurant:tuple, destination: tuple) -> float:
        return math.dist(source, restaurant) / meters_per_minute + math.dist(restaurant, destination) / meters_per_minute

   # check if the courier can take the order
    def can_assign(self, order : Order) -> bool:
        # if the ready time < courier's off time
        if order.ready_time < self.on_time:
            return False
        if order.ready_time > self.off_time:
            return False

    # assign an order to courier
    def assign_order(self, order: Order) -> bool:
        # update available time for the next assignment
        _, restaurant_x, restaurant_y = restaurants[restaurants['restaurant'] == order.restaurant].values[0]
        if self.can_assign(order):
            self.orders.append(order)
            self.current_time += self.calculate_travel_time(self.current_position, (restaurant_x, restaurant_y), orders.destination)

In [111]:
class DeliveryRouting:
    def __init__(self, instance_dir : str ):
        orders, restaurants, couriers, instanceparams, locations = read_instance_information(instance_dir)

        self.orders = [Order(order) for order in orders.to_dict(orient = 'records')]
        self.restaurants = restaurants.to_dict(orient = 'records')
        self.couriers = [Courier(courier, self.restaurants) for courier in couriers.to_dict(orient = 'records')]
        self.unassigned_orders = self.orders.copy()

        self.instance_params = instanceparams.to_dict(orient = 'records')[0]

        self.meters_per_minute, self.pickup_service_minutes, self.dropoff_service_minutes, \
            self.target_click_to_door, self.maximum_click_to_door, self.pay_per_order,\
            self.guaranteed_pay_per_hour = self.instance_params.values()


    def random_initialization(self):
        pass

In [112]:
# orders,restaurants,couriers,instanceparams,locations

In [113]:
instance_dir = 'data/0o50t75s1p100'
dr = DeliveryRouting(instance_dir)
dr.unassigned_orders

**Funtion that calculate bundel size Z(t)**

In [ ]:
def get_bundel_size():
    ...

**Function that calculate k(r,t)**<br>
k(r,t) is the number of couriers who are available at restaurant r

In [ ]:
def get_available_courier(t,r):
    ...

**Route object**<br>
A route is a bundle,i.e: a list of ordered orders

In [ ]:
class Route(object):
    def __init__:
    
    # calculate total travel time from 1st destination to the last destination of the route
    def total_travel_time():

    # calculate total service delay    
    def total_service_delay():
        #<to be discuss>

    # calculate total_click_to_door 
    def total_click_to_door():
        #<to be discuss>

    # calculate route efficiency: travel time per order:
    def route_efficiency():

    # calculate route cost
    def route_cost():
        #route_cost = total_travel_time + beta*total_service_delay + gamma*total_click_to_door


# Implement procedure 1